In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=bf2a9da6f818ed5b85b405e1aab04b4fb6fb2cfa8c541ac457d4bfa0d149299f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
from fpdf import FPDF
import pandas as pd
import matplotlib.pyplot as plt

# Sample DataFrames from oracle live sql outputs
departments = pd.DataFrame({
    'dept_id': [1, 2, 3],
    'dept_name': ['Sales', 'Marketing', 'HR'],
    'location': ['New York', 'Los Angeles', 'Chicago']
})

employees = pd.DataFrame({
    'emp_id': [101, 102, 103, 104],
    'emp_name': ['Alice', 'Bob', 'Charlie', 'David'],
    'dept_id': [1, 1, 2, 3],
    'hire_date': ['2023-02-10', '2022-03-15', '2023-05-20', '2021-06-18'],
    'salary': [60000, 55000, 58000, 52000]
})

sales = pd.DataFrame({
    'sale_id': [1001, 1002, 1003],
    'emp_id': [101, 102, 103],
    'product_category': ['Electronics', 'Furniture', 'Electronics'],
    'amount': [12000, 8000, 9000],
    'location': ['New York', 'New York', 'Los Angeles'],
    'sale_date': ['2023-07-12', '2022-09-25', '2023-08-10']
})

# PDF Report
class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'OLAP SQL Report', border=False, ln=True, align='C')
        self.ln(5)

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.set_fill_color(200, 220, 255)
        self.cell(0, 10, title, ln=True, fill=True)
        self.ln(2)

    def table_from_df(self, df):
        self.set_font('Arial', '', 10)
        col_widths = [self.w / len(df.columns)] * len(df.columns)
        for col in df.columns:
            self.cell(col_widths[0], 8, str(col), border=1)
        self.ln()
        for _, row in df.iterrows():
            for item in row:
                self.cell(col_widths[0], 8, str(item), border=1)
            self.ln()
        self.ln(5)

pdf = PDFReport()
pdf.add_page()

pdf.chapter_title("1. Departments Table")
pdf.table_from_df(departments)

pdf.chapter_title("2. Employees Table")
pdf.table_from_df(employees)

pdf.chapter_title("3. Sales Table")
pdf.table_from_df(sales)

# Creating a chart for total salary by department
merged = pd.merge(employees, departments, on='dept_id')
salary_by_dept = merged.groupby('dept_name')['salary'].sum()
plt.figure(figsize=(5,3))
salary_by_dept.plot(kind='bar', color='skyblue', title='Total Salary by Department')
plt.ylabel('Salary')
plt.tight_layout()
plt.savefig('salary_by_dept.png')
plt.close()

# Inserting chart into PDF
pdf.add_page()
pdf.chapter_title("Chart: Total Salary by Department")
pdf.image('salary_by_dept.png', x=10, y=None, w=180)

# Saving PDF
pdf_path = "OLAP_SQL_Report.pdf"
pdf.output(pdf_path)
pdf_path


'OLAP_SQL_Report.pdf'